<a href="https://colab.research.google.com/github/cleysonl/ML_Bootcamp_CLL/blob/master/Data_Processing_and_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Processing and Analysis**

In [0]:
import datetime
import random
from random import randrange
import numpy as np
import pandas as pd

In [0]:
def _random_date(start,date_count):
    """This function generates a random date based on params
    Args:
        start (date object): the base date
        date_count (int): number of dates to be generated
    Returns:
        list of random dates

    """
    current = start
    while date_count > 0:
        curr = current + datetime.timedelta(days=randrange(42))
        yield curr
        date_count-=1
        
        

def generate_sample_data(row_count=100):
    """This function generates a random transaction dataset
    Args:
        row_count (int): number of rows for the dataframe
    Returns:
        a pandas dataframe

    """

    # sentinels
    startDate = datetime.datetime(2016, 1, 1, 13)
    serial_number_sentinel = 1000
    user_id_sentinel = 5001
    product_id_sentinel = 101
    price_sentinel = 2000

    # base list of attributes
    data_dict = {
        'Serial No':
        np.arange(row_count) + serial_number_sentinel,
        'Date':
        np.random.permutation(
            pd.to_datetime([
                x.strftime("%d-%m-%Y")
                for x in _random_date(startDate, row_count)
            ]).date),
        'User ID':
        np.random.permutation(
            np.random.randint(0, row_count, size=int(row_count / 10)) +
            user_id_sentinel).tolist() * 10,
        'Product ID':
        np.random.permutation(
            np.random.randint(0, row_count, size=int(row_count / 10)) +
            product_id_sentinel).tolist() * 10,
        'Quantity Purchased':
        np.random.permutation(np.random.randint(1, 42, size=row_count)),
        'Price':
        np.round(
            np.abs(np.random.randn(row_count) + 1) * price_sentinel,
            decimals=2),
        'User Type':
        np.random.permutation(
            [chr(random.randrange(97, 97 + 3 + 1)) for i in range(row_count)])
    }

    # introduce missing values
    for index in range(int(np.sqrt(row_count))):
        data_dict['Price'][np.argmax(
            data_dict['Price'] == random.choice(data_dict['Price']))] = np.nan
        data_dict['User Type'][np.argmax(
            data_dict['User Type'] == random.choice(
                data_dict['User Type']))] = np.nan
        data_dict['Date'][np.argmax(
            data_dict['Date'] == random.choice(data_dict['Date']))] = np.nan
        data_dict['Product ID'][np.argmax(data_dict['Product ID'] == random.
                                          choice(data_dict['Product ID']))] = 0
        data_dict['Serial No'][np.argmax(data_dict['Serial No'] == random.
                                         choice(data_dict['Serial No']))] = -1
        data_dict['User ID'][np.argmax(data_dict['User ID'] == random.choice(
            data_dict['User ID']))] = -101

    # create data frame
    df = pd.DataFrame(data_dict)

    return df

## **Import dependencies**

In [0]:
import numpy as np
import pandas as pd
from IPython.display import display
from sklearn import preprocessing

pd.options.mode.chained_assignment = None

## **Generate dataset**

In [4]:
# Generate a dataset with 1000 rows
df = generate_sample_data(row_count=1000)
df.shape

(1000, 7)

### **Analyze generated Dataset**

In [5]:
df.head()

,Serial No,Date,User ID,Product ID,Quantity Purchased,Price,User Type
0,1000,NaN,-101,0,10,7.10,n
1,1001,2016-05-01,5426,726,32,834.55,n
2,1002,NaN,5222,936,27,2046.63,n
3,1003,NaN,5716,347,21,1206.54,n
4,1004,NaN,5065,604,19,2824.19,n


**Dataframe stats**

In [6]:
print('Number of rows:', df.shape[0])

Number of rows: 1000


In [7]:
print('Number of columns:', df.shape[1])

Number of columns: 7


In [8]:
print('Column data types: \n', df.dtypes)

Column data types: 
 Serial No               int64
Date                   object
User ID                 int64
Product ID              int64
Quantity Purchased      int64
Price                 float64
User Type              object
dtype: object


In [9]:
print('Columns with missing values:', df.columns[df.isnull().any()].tolist())

Columns with missing values: ['Date', 'Price']


In [10]:
print('Number of rows with Missing values:', len(pd.isnull(df).any(1).nonzero()[0].tolist()))

Number of rows with Missing values: 62


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  """Entry point for launching an IPython kernel.


**General Stats**

In [11]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
Serial No             1000 non-null int64
Date                  969 non-null object
User ID               1000 non-null int64
Product ID            1000 non-null int64
Quantity Purchased    1000 non-null int64
Price                 969 non-null float64
User Type             1000 non-null object
dtypes: float64(1), int64(4), object(2)
memory usage: 54.8+ KB
None


In [12]:
print(df.describe())

         Serial No      User ID   Product ID  Quantity Purchased        Price
count  1000.000000  1000.000000  1000.000000         1000.000000   969.000000
mean   1452.193000  5510.703000   565.698000           21.015000  2246.807296
std     382.054228   340.229183   279.269601           11.743282  1606.744565
min      -1.000000  -101.000000     0.000000            1.000000     7.100000
25%    1225.750000  5288.000000   325.000000           11.000000   898.650000
50%    1481.500000  5517.500000   524.000000           20.000000  2001.540000
75%    1738.250000  5744.750000   807.000000           31.000000  3243.250000
max    1999.000000  6000.000000  1091.000000           41.000000  7626.880000


 ### **Standardize Columns**

In [13]:
# list all columns
print("Dataframe columns>\n{}".format(df.columns.tolist()))

Dataframe columns>
['Serial No', 'Date', 'User ID', 'Product ID', 'Quantity Purchased', 'Price', 'User Type']


In [0]:
# change all columns names to lowercase-snakecased column names in python
def cleanup_column_names(df, rename_dict={}, do_inplace=True):
  if not rename_dict:
    return df.rename(columns={col:col.lower().replace(' ','_') for col in df.columns.values.tolist()}, inplace=True)
  else:
    return df.rename(columns = rename_dict, inplace=do_inplace)

In [0]:
cleanup_column_names(df)

In [16]:
print("Dataframe columns:\n{}".format(df.columns.tolist()))

Dataframe columns:
['serial_no', 'date', 'user_id', 'product_id', 'quantity_purchased', 'price', 'user_type']


### **Basic Manipulation**

**Sort basic specific atributes**

In [18]:
# Ascending for serial_no and descending for price
display(df.sort_values(['serial_no','price'], ascending=[True, False]).head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
601,-1,2016-01-23,5426,726,34,5830.82,d
866,-1,2016-05-01,5896,974,33,5592.77,c
222,-1,2016-05-02,5615,122,27,4047.99,b
852,-1,2016-01-28,5927,211,32,3961.71,d
600,-1,2016-01-15,5306,192,31,3774.54,c


**Reorder columns**

In [19]:
display(df[['serial_no','date','user_id','user_type',
              'product_id','quantity_purchased','price']].head())

,serial_no,date,user_id,user_type,product_id,quantity_purchased,price
0,1000,NaN,-101,n,0,10,7.10
1,1001,2016-05-01,5426,n,726,32,834.55
2,1002,NaN,5222,n,936,27,2046.63
3,1003,NaN,5716,n,347,21,1206.54
4,1004,NaN,5065,n,604,19,2824.19


**Select Attributes**

In [21]:
# Using column index print 10 values from column at index 3
print(df.iloc[:,3].values[0:10])

[  0 726 936 347 604 182 865 237 900 213]


In [24]:
# Using column name print 10 values of quantity_purchased
print(df.quantity_purchased.values[0:10])

[10 32 27 21 19 14 14  8 37 21]


In [25]:
# Using datatype, print 10 values of columns with data type float
print(df.select_dtypes(include=['float64']).values[:10,0])

[   7.1   834.55 2046.63 1206.54 2824.19 1211.12 2268.52 4468.28 2987.57
 1555.05]


**Select rows**

In [26]:
# using row index
display(df.iloc[[10,501,20]])

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
10,1010,2016-01-18,5371,796,10,701.83,n
501,1501,2016-01-26,5426,726,21,722.10,a
20,1020,2016-01-26,5346,531,19,3127.34,n


In [27]:
# excluding specific rows
display(df.drop([0,24,51], axis=0).head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
1,1001,2016-05-01,5426,726,32,834.55,n
2,1002,NaN,5222,936,27,2046.63,n
3,1003,NaN,5716,347,21,1206.54,n
4,1004,NaN,5065,604,19,2824.19,n
5,1005,2016-07-02,5699,182,14,1211.12,n


In [28]:
# Conditional filtering
# Quantity_purchased > 25
display(df[df.quantity_purchased>25].head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
1,1001,2016-05-01,5426,726,32,834.55,n
2,1002,NaN,5222,936,27,2046.63,n
8,1008,2016-08-02,6000,900,37,2987.57,n
12,1012,2016-02-01,5136,395,41,1183.75,n
14,1014,2016-11-02,5549,163,30,2358.50,n


In [29]:
# Offset from top
display(df[100:].head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
100,1100,NaN,5306,192,9,2558.69,d
101,1101,2016-01-28,5426,726,33,2385.22,a
102,1102,2016-01-25,5222,936,41,3261.19,a
103,1103,2016-01-13,5716,347,36,3272.22,d
104,1104,2016-09-01,5065,604,10,223.15,d


In [30]:
#offset from Bottom
display(df[-10:].head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
990,1990,2016-06-01,5427,708,21,3791.00,b
991,1991,2016-01-16,5500,652,9,188.01,b
992,1992,2016-01-20,5781,351,18,785.80,c
993,1993,2016-03-02,5853,379,20,245.14,d
994,1994,2016-12-01,5565,641,28,4813.95,a


**Type casting**

In [32]:
# existing datatypes
df.dtypes

serial_no               int64
date                   object
user_id                 int64
product_id              int64
quantity_purchased      int64
price                 float64
user_type              object
dtype: object

In [33]:
# set datatime as dtype for date column
df['date'] = pd.to_datetime(df.date)
print(df.dtypes)

serial_no                      int64
date                  datetime64[ns]
user_id                        int64
product_id                     int64
quantity_purchased             int64
price                        float64
user_type                     object
dtype: object


**Map/Apply Functionality**

In [0]:
def expand_user_type(u_type):
  if u_type in ['a','b']:
    return 'new'
  elif u_type == 'c':
    return 'existing'
  elif u_type == 'd':
    return 'loyal_existing'
  else:
    return 'error'

In [35]:
# Map user Type to user class
df['user_class'] = df['user_type'].map(expand_user_type)
display(df.tail())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class
995,1995,2016-07-01,5032,900,15,1300.10,c,existing
996,1996,2016-10-02,5288,621,25,4295.21,b,new
997,1997,2016-09-02,5203,461,38,1037.14,a,new
998,1998,2016-01-16,5016,915,8,648.07,b,new
999,1999,2016-06-01,5180,493,37,369.54,b,new
